In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
!wget https://dlcdn.apache.org/spark/spark-3.3.0/spark-3.3.0-bin-hadoop2.tgz

--2022-10-11 08:10:10--  https://dlcdn.apache.org/spark/spark-3.3.0/spark-3.3.0-bin-hadoop2.tgz
Resolving dlcdn.apache.org (dlcdn.apache.org)... 151.101.2.132, 2a04:4e42::644
Connecting to dlcdn.apache.org (dlcdn.apache.org)|151.101.2.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 274079476 (261M) [application/x-gzip]
Saving to: ‘spark-3.3.0-bin-hadoop2.tgz’

spark-3.3.0-bin-had 100%[===================>] 261.38M   269MB/s    in 1.0s    

2022-10-11 08:10:12 (269 MB/s) - ‘spark-3.3.0-bin-hadoop2.tgz’ saved [274079476/274079476]



In [ ]:
!tar xf spark-3.3.0-bin-hadoop2.tgz

In [ ]:
!pip install findspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "spark-3.3.0-bin-hadoop2"

In [ ]:
import findspark
findspark.init()

In [ ]:
from pyspark.sql.types import IntegerType

In [ ]:
findspark.find()

'spark-3.3.0-bin-hadoop2'

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local").appName("Colab").config("spark.ui.port",'4050').getOrCreate()

In [ ]:
data = spark.read.csv("/content/drive/MyDrive/CSCI316/data.csv", inferSchema=True, header=True)

In [ ]:
data.show(10)

+-------+---------+---------+-----------+---------------+----------+--------+-----------+-----+---------+--------------------+----------+--------------+----------+-------------------+----------+----------+--------------------+------------------+--------------------+--------+----------+-----+-----------+----------------+--------------+----------------------+----------------------+--------+-------+---------+----------+---------+-------------------+---------+-------------+-----------+---------------+---------------+-------------+------------------+----------+-----------------------+------------+---------------+------------+------------------+--------------------------+---------------------------+-----------+----------------+----------------+---------+-------------------------+--------------+------------+-----------+-----------+----------+-----------+-----------+------------------+------------+-------+-----------+-----------+----------+--------+----------------+------+-----------+---------

In [ ]:
hi = data.take(1)

In [ ]:
type(data)

pyspark.sql.dataframe.DataFrame

In [ ]:
#check for the missing values
from pyspark.sql.functions import isnan, when, count, col
data.select([count(when(col(c).isNull(), c)).alias(c) for c in data.columns]).show()

+---+---------+---------+-----------+---------------+----+--------+-----------+-----+---------+---------+----------+--------------+----------+-------------------+-------+----------+------+-------+-----+--------+----------+---+-----------+----------------+--------------+----------------------+----------------------+--------+-------+---------+----------+---------+-------------------+---------+-------------+-----------+---------------+---------------+-------------+------------------+----------+-----------------------+------------+---------------+------------+------------------+--------------------------+---------------------------+-----------+----------------+----------------+---------+-------------------------+--------------+------------+-----------+-----------+----------+-----------+-----------+------------------+------------+-------+-----------+-----------+----------+--------+----------------+------+-----------+------------+-----------+
| id|member_id|loan_amnt|funded_amnt|funded_amnt_

In [ ]:
#get the percentage of null values 
amount_missing_df = data.select([((count(when(col(c).isNull(), c))/data.count())*100).alias(c) for c in data.columns])
amount_missing_df.show()
#from the table below, i think we can drop the columns that consist of 40% or higher null values.
#things that need to be dropped: desc, annual_inc_joint, dti_joint, verification_status_joint,
# maybe do research: mths_since_last_delinq, mths_since_last_record, mths_since_last_major_derog, open_acc_6m, open_il_6m

+---+---------+---------+-----------+---------------+----+--------+-----------+-----+---------+------------------+----------+--------------+----------+-------------------+-------+----------+-----------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------------------+----------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------------+------------------+--------------------+-----------------+--------------------+--------------------------+---------------------------+--------------------+--------------------+----------------+-----------------+-------------------------+--------------------+--

In [ ]:
#Show the types
data.dtypes

[('id', 'int'),
 ('member_id', 'int'),
 ('loan_amnt', 'int'),
 ('funded_amnt', 'int'),
 ('funded_amnt_inv', 'double'),
 ('term', 'string'),
 ('int_rate', 'double'),
 ('installment', 'double'),
 ('grade', 'string'),
 ('sub_grade', 'string'),
 ('emp_title', 'string'),
 ('emp_length', 'string'),
 ('home_ownership', 'string'),
 ('annual_inc', 'string'),
 ('verification_status', 'string'),
 ('issue_d', 'string'),
 ('pymnt_plan', 'string'),
 ('desc', 'string'),
 ('purpose', 'string'),
 ('title', 'string'),
 ('zip_code', 'string'),
 ('addr_state', 'string'),
 ('dti', 'string'),
 ('delinq_2yrs', 'string'),
 ('earliest_cr_line', 'string'),
 ('inq_last_6mths', 'string'),
 ('mths_since_last_delinq', 'string'),
 ('mths_since_last_record', 'string'),
 ('open_acc', 'string'),
 ('pub_rec', 'string'),
 ('revol_bal', 'string'),
 ('revol_util', 'string'),
 ('total_acc', 'string'),
 ('initial_list_status', 'string'),
 ('out_prncp', 'string'),
 ('out_prncp_inv', 'string'),
 ('total_pymnt', 'string'),
 ('t

In [ ]:
data.groupby('emp_title').count().show()

+--------------------+-----+
|           emp_title|count|
+--------------------+-----+
|      John Wiley Jr.|    1|
|baylor college of...|    2|
|         Houston ISD|   14|
|      Real Mex Foods|    2|
| City of Tulsa, Okla|    1|
|SOUTHERN WINE & S...|    1|
|Medversant Techno...|    2|
|   Porter Scott, APC|    2|
|         PCM Studios|    1|
|Smith Ironworks, Inc|    1|
|           sams club|    9|
|morgan stanley sm...|    4|
|G4S Secure Soluti...|    1|
|           Blue Nile|    1|
|  General Motors LLC|    3|
|        DoD Civilian|    2|
|SLAC National Acc...|    1|
|atlantic tomorrow...|    1|
|       DemandTec Inc|    1|
|       Famous Daves |    1|
+--------------------+-----+
only showing top 20 rows



In [ ]:
data.groupby('id').count().show()

+-------+-----+
|     id|count|
+-------+-----+
|1068322|    1|
|1060895|    1|
|1058372|    1|
|1057878|    1|
|1045554|    1|
|1030666|    1|
|1024037|    1|
|1023029|    1|
|1020097|    1|
|1016416|    1|
|1011733|    1|
|1003031|    1|
| 998884|    1|
| 999834|    1|
| 974413|    1|
| 994355|    1|
| 994660|    1|
| 995357|    1|
| 991309|    1|
| 987080|    1|
+-------+-----+
only showing top 20 rows



In [ ]:
#Check distinct value
data.groupby("term").count().show()

+----------+------+
|      term| count|
+----------+------+
| 36 months|600221|
| 60 months|255748|
+----------+------+



In [ ]:
data.groupby("grade").count().show()

+-----+------+
|grade| count|
+-----+------+
|    F| 21328|
|    E| 66448|
|    B|247998|
|    D|132802|
|    C|236855|
|    A|145665|
|    G|  4873|
+-----+------+



In [ ]:
data.groupby("sub_grade").count().show()

+---------+-----+
|sub_grade|count|
+---------+-----+
|       D5|20208|
|       F2| 4979|
|       B4|54116|
|       A2|22106|
|       E4|11011|
|       B2|47589|
|       C3|48337|
|       D1|34667|
|       C4|46941|
|       F1| 6702|
|       D3|25289|
|       F5| 2380|
|       G2| 1259|
|       B1|44002|
|       B3|54958|
|       E5| 8919|
|       C5|39532|
|       G3|  877|
|       A4|33991|
|       F4| 3168|
+---------+-----+
only showing top 20 rows



In [ ]:
data.groupby("emp_length").count().show()

+----------+------+
|emp_length| count|
+----------+------+
|   9 years| 33462|
|   5 years| 53812|
|    1 year| 54855|
|       n/a| 43061|
|   2 years| 75985|
|   7 years| 43204|
|   8 years| 42421|
|   4 years| 50643|
|   6 years| 41446|
|   3 years| 67392|
| 10+ years|282090|
|  < 1 year| 67597|
| reactors"|     1|
+----------+------+



In [ ]:
data.groupby("home_ownership").count().show()

+--------------+------+
|home_ownership| count|
+--------------+------+
|           OWN| 84136|
|          RENT|342535|
|      MORTGAGE|429105|
|           ANY|     3|
|         OTHER|   144|
|          NONE|    45|
|       2 years|     1|
+--------------+------+



In [ ]:
data.groupby("verification_status").count().show()

+-------------------+------+
|verification_status| count|
+-------------------+------+
|           Verified|280049|
|    Source Verified|318177|
|       Not Verified|257742|
|              38000|     1|
+-------------------+------+



In [ ]:
data.groupby("issue_d").count().show()

+----------+-----+
|   issue_d|count|
+----------+-----+
|01-12-2010| 1266|
|01-06-2011| 1826|
|01-06-2012| 3790|
|01-11-2007|   37|
|01-07-2010| 1119|
|01-04-2009|  290|
|01-10-2014|37442|
|01-06-2007|    1|
|01-06-2013|10663|
|01-12-2009|  598|
|01-11-2009|  602|
|01-03-2010|  737|
|01-10-2009|  545|
|01-07-2013|11632|
|01-08-2011| 1925|
|01-09-2011| 2062|
|01-01-2011| 1368|
|01-03-2009|  276|
|01-05-2011| 1701|
|01-02-2010|  627|
+----------+-----+
only showing top 20 rows



In [ ]:
data.groupby("desc").count().show()

+--------------------+-----+
|                desc|count|
+--------------------+-----+
|  Borrower added ...|    1|
|  Borrower added ...|    1|
|  Borrower added ...|    1|
|  Borrower added ...|    1|
|  Borrower added ...|    1|
|  Borrower added ...|    1|
|  Borrower added ...|    1|
| Requesting a loa...|    1|
|  Borrower added ...|    1|
|  Borrower added ...|    1|
|  Borrower added ...|    1|
|  Borrower added ...|    1|
|  Borrower added ...|    1|
|  Borrower added ...|    1|
|  Borrower added ...|    1|
|  Borrower added ...|    1|
|  Borrower added ...|    1|
|  Borrower added ...|    1|
|  Borrower added ...|    1|
|  Borrower added ...|    1|
+--------------------+-----+
only showing top 20 rows



In [ ]:
data.groupby("purpose").count().show()

+--------------------+-----+
|             purpose|count|
+--------------------+-----+
| I have eliminate...|    1|
| I am not comfort...|    1|
| my cell is on a ...|    1|
| MD""... approx. ...|    1|
| and have never h...|    1|
|<br/><br/>Lending...|    1|
| which was a swin...|    1|
| usually doubling...|    1|
| as long as we pa...|    1|
| and have a solid...|    1|
| but want to make...|    1|
| the grass is in ...|    1|
|        increasingly|    1|
| which is $250 a ...|    1|
| I have the means...|    1|
| I will receive t...|    1|
| I will be able t...|    1|
| at which point I...|    1|
| i wasn't even bo...|    1|
| it is a historic...|    1|
+--------------------+-----+
only showing top 20 rows



In [ ]:
data.groupby("title").count().show()

+--------------------+-----+
|               title|count|
+--------------------+-----+
|          Debit Free|   12|
|        Debt reducer|    4|
|        HOME REMODEL|    6|
|My debt consolida...|   20|
|Loan - Home Impro...|    1|
|Debt Consolidatio...|   17|
|         Better rate|    3|
|                TC's|    1|
|2011 GulfStream S...|    1|
|              travel|    9|
|   Debit Colsidation|    1|
|          Car Payoff|   11|
|             MY LOAN|   39|
|        sears payoff|    3|
|Increase my cash ...|    1|
|        Debt Removal|    8|
|Home improvement ...|    1|
|Credit Card Free ...|    1|
|Credit Card Reduc...|   30|
|   Credit Card Combo|    2|
+--------------------+-----+
only showing top 20 rows



In [ ]:
data.groupby("addr_state").count().show()

+--------------------+-----+
|          addr_state|count|
+--------------------+-----+
|Helping Kenya's D...|    1|
|               223xx|    1|
|175 (total projec...|    1|
| financially I ma...|    1|
|     and businessman|    1|
| not paid for by ...|    1|
|         etc.  First|    1|
|                  AZ|19685|
|                  SC|10301|
| we both stand to...|    1|
| I will let this ...|    1|
| Individual Schoo...|    1|
|                  LA|10185|
|                  MN|15420|
|               499xx|    1|
|               850xx|    1|
|Advocate business...|    1|
|               951xx|    1|
|               662xx|    1|
| I ALWAYS pay my ...|    1|
+--------------------+-----+
only showing top 20 rows



In [ ]:
data.groupby("earliest_cr_line").count().show()

+--------------------+-----+
|    earliest_cr_line|count|
+--------------------+-----+
|          01-08-2004| 4447|
|          01-11-2005| 2955|
|          01-10-2001| 5922|
|          01-06-1975|  151|
|          01-12-1989| 1446|
|          01-12-1977|  275|
|               223xx|    1|
|          01-12-2010|  567|
|          01-01-2002| 4281|
| and pay off the ...|    1|
|          01-06-2011|  534|
|          01-11-2007| 1651|
|          01-03-1995| 2805|
|          01-02-1999| 3786|
|          01-02-1993| 1697|
|          01-02-1998| 3005|
|          01-10-1978|  331|
|               895xx|    1|
|                2010|    1|
|          01-07-2010|  623|
+--------------------+-----+
only showing top 20 rows



In [ ]:
data.groupby("initial_list_status").count().show()

+--------------------+-----+
| initial_list_status|count|
+--------------------+-----+
|                8433|    1|
|               17.42|    1|
|                   7|    5|
|                7013|    1|
|                  AZ|    1|
|          01-06-2006|    1|
|                  15|    4|
|               11534|    1|
|                5656|    1|
|                5076|    1|
|                6.18|    1|
|                  29|    1|
|                68.1|    1|
|          01-09-1984|    1|
|                8828|    1|
|               26050|    1|
|000 is a motorcyc...|    1|
|                  30|    1|
|                94.6|    1|
|                   8|    1|
+--------------------+-----+
only showing top 20 rows



In [ ]:
data.groupby("last_pymnt_d").count().show()

+------------+-----+
|last_pymnt_d|count|
+------------+-----+
|  01-11-2015|10692|
|  01-12-2010|  248|
|      600.07|    1|
|  01-06-2012|  816|
|  01-06-2011|  373|
|     4330.08|    1|
|  01-07-2010|  181|
|  01-04-2009|   43|
|  01-10-2014| 7671|
|    16332.89|    1|
|  01-06-2013| 1971|
|     3569.48|    1|
|    14484.55|    1|
|  01-12-2009|   93|
|     1788.95|    1|
|  01-11-2009|   58|
|  01-04-2015|11101|
|  01-03-2010|  134|
|    14006.82|    1|
|  01-10-2009|   61|
+------------+-----+
only showing top 20 rows



In [ ]:
data.groupby("next_pymnt_d").count().show()

+------------+-----+
|next_pymnt_d|count|
+------------+-----+
|     2509.98|    1|
|  01-12-2010|    2|
|       14000|    1|
|    13352.45|    1|
|  01-06-2012|    1|
|      854.08|    1|
|     8905.29|    1|
|  01-07-2010|    1|
|        0.59|    1|
|     4234.33|    1|
|    20198.65|    1|
|      414.65|    1|
|           3|    1|
|      282.98|    1|
|      246.35|    1|
|       94.06|    1|
|      751.86|    1|
|           8|    1|
|      368.79|    1|
|       22.12|    1|
+------------+-----+
only showing top 20 rows



In [ ]:
data.groupby("last_credit_pull_d").count().show()

+------------------+-----+
|last_credit_pull_d|count|
+------------------+-----+
|        01-11-2015|11386|
|        01-12-2010|  113|
|          28195.63|    1|
|        01-06-2012|  352|
|        01-06-2011|  168|
|             14000|    1|
|        01-07-2010|  107|
|           6016.66|    1|
|        01-04-2009|   16|
|        01-10-2014| 3343|
|           5181.62|    1|
|           6315.45|    1|
|        01-06-2007|   12|
|             30.42|    1|
|             68.69|    1|
|          23473.88|    1|
|          19465.96|    1|
|             11500|    1|
|            369.52|    1|
|           1499.99|    1|
+------------------+-----+
only showing top 20 rows



In [ ]:
data.groupby("application_type").count().show()

+----------------+-----+
|application_type|count|
+----------------+-----+
|         5802.38|    1|
|      01-06-2011|    1|
|      01-06-2012|    1|
|      01-07-2010|    2|
|         1011.14|    1|
|           18131|    1|
|      01-01-2011|    1|
|      01-05-2011|    2|
|      01-03-2015|    1|
|               0|   63|
|            null|   57|
|          594.78|    1|
|               f|    1|
|      01-07-2012|    1|
|      01-12-2012|    2|
|           21.38|    1|
|      01-01-2012|    1|
|          1466.3|    1|
|      01-02-2012|    2|
|      01-09-2010|    1|
+----------------+-----+
only showing top 20 rows



In [ ]:
#if it is allowed: 
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer(inputCol='term', outputCol='term_numeric')
indexer_fitted = indexer.fit(data)
df_indexed = indexer_fitted.transform(data)
df_indexed.show()

+-------+---------+---------+-----------+---------------+----------+--------+-----------+-----+---------+--------------------+----------+--------------+----------+-------------------+----------+----------+--------------------+------------------+--------------------+--------+----------+-----+-----------+----------------+--------------+----------------------+----------------------+--------+-------+---------+----------+---------+-------------------+---------+-------------+-----------+---------------+---------------+-------------+------------------+----------+-----------------------+------------+---------------+------------+------------------+--------------------------+---------------------------+-----------+----------------+----------------+---------+-------------------------+--------------+------------+-----------+-----------+----------+-----------+-----------+------------------+------------+-------+-----------+-----------+----------+--------+----------------+------+-----------+---------

In [ ]:
#to replace null with n/a
from pyspark.sql.functions import col,lit
columns_string_replace = ["emp_title","desc","zip_code","addr_state","dti","delinq_2yrs","earliest_cr_line","purpose","title","initial_list_status","application_type","inq_last_6mths","mths_since_last_delinq",
                          "mths_since_last_record","open_acc","pub_rec","revol_bal","revol_util","total_acc","initial_list_status","out_prncp","out_prncp_inv","total_pymnt","total_pymnt_inv","total_rec_prncp", 
                          "total_rec_prncp","total_rec_int","total_rec_late_fee","recoveries","collection_recovery_fee","last_pymnt_d","last_pymnt_amnt","next_pymnt_d","last_credit_pull_d","collections_12_mths_ex_med",
                          'mths_since_last_major_derog', 'policy_code', 'application_type', 'annual_inc_joint', 'dti_joint', 'verification_status_joint', 'acc_now_delinq', 'tot_coll_amt' , 'tot_cur_bal', 'open_acc_6m', 'open_il_6m', 
                          'open_il_12m', 'open_il_24m', 'mths_since_rcnt_il', 'total_bal_il', 'il_util', 'open_rv_12m', 'open_rv_24m', 'max_bal_bc', 'all_util', 'total_rev_hi_lim', 'inq_fi', 'total_cu_tl' ]

for i in columns_string_replace:
  data = data.withColumn(i, when(data[i].isNull(), lit("n/a")).otherwise(data[i]))

data = data.withColumn("inq_last_12m", when(data["inq_last_12m"].isNull(), lit(0)).otherwise(data["inq_last_12m"]))

In [ ]:
data.show()

+-------+---------+---------+-----------+---------------+----------+--------+-----------+-----+---------+--------------------+----------+--------------+----------+-------------------+----------+----------+--------------------+------------------+--------------------+--------+----------+-----+-----------+----------------+--------------+----------------------+----------------------+--------+-------+---------+----------+---------+-------------------+---------+-------------+-----------+---------------+---------------+-------------+------------------+----------+-----------------------+------------+---------------+------------+------------------+--------------------------+---------------------------+-----------+----------------+----------------+---------+-------------------------+--------------+------------+-----------+-----------+----------+-----------+-----------+------------------+------------+-------+-----------+-----------+----------+--------+----------------+------+-----------+---------

In [ ]:
def find_outliers(df):

    # Identifying the numerical columns in a spark dataframe
    numeric_columns = [column[0] for column in df.dtypes if column[1]=='int']

    # Using the `for` loop to create new columns by identifying the outliers for each feature
    for column in numeric_columns:

        less_Q1 = 'less_Q1_{}'.format(column)
        more_Q3 = 'more_Q3_{}'.format(column)
        Q1 = 'Q1_{}'.format(column)
        Q3 = 'Q3_{}'.format(column)

        # Q1 : First Quartile ., Q3 : Third Quartile
        Q1 = df.approxQuantile(column,[0.25],relativeError=0)
        Q3 = df.approxQuantile(column,[0.75],relativeError=0)
        
        # IQR : Inter Quantile Range
        # We need to define the index [0], as Q1 & Q3 are a set of lists., to perform a mathematical operation
        # Q1 & Q3 are defined seperately so as to have a clear indication on First Quantile & 3rd Quantile
        IQR = Q3[0] - Q1[0]
        
        #selecting the data, with -1.5*IQR to + 1.5*IQR., where param = 1.5 default value
        less_Q1 =  Q1[0] - 1.5*IQR
        more_Q3 =  Q3[0] + 1.5*IQR
        
        isOutlierCol = 'is_outlier_{}'.format(column)
        
        df = df.withColumn(isOutlierCol, when((df[column] > more_Q3) | (df[column] < less_Q1), 1).otherwise(0))
    

    # Selecting the specific columns which we have added above, to check if there are any outliers
    selected_columns = [column for column in df.columns if column.startswith("is_outlier")]

    # Adding all the outlier columns into a new colum "total_outliers", to see the total number of outliers
    df = df.withColumn('total_outliers',sum(df[column] for column in selected_columns))

    # Dropping the extra columns created above, just to create nice dataframe., without extra columns
    df = df.drop(*[column for column in df.columns if column.startswith("is_outlier")])

    return df

In [ ]:
new_data = find_outliers(data)
#new_data.show()

new_data_with_no_outliers = new_data.filter(new_data['total_Outliers']<=1)
new_data_with_no_outliers = new_data_with_no_outliers.select(*data.columns)

new_data_with_no_outliers.show()

+-------+---------+---------+-----------+---------------+----------+--------+-----------+-----+---------+--------------------+----------+--------------+----------+-------------------+----------+----------+--------------------+------------------+--------------------+--------+----------+-----+-----------+----------------+--------------+----------------------+----------------------+--------+-------+---------+----------+---------+-------------------+---------+-------------+-----------+---------------+---------------+-------------+------------------+----------+-----------------------+------------+---------------+------------+------------------+--------------------------+---------------------------+-----------+----------------+----------------+---------+-------------------------+--------------+------------+-----------+-----------+----------+-----------+-----------+------------------+------------+-------+-----------+-----------+----------+--------+----------------+------+-----------+---------

Drop the missing values in the default_ind

In [ ]:
delete_null_predict = new_data_with_no_outliers.filter(new_data_with_no_outliers.default_ind.isNotNull())

In [ ]:
delete_null_predict.select([count(when(col(c).isNull(), c)).alias(c) for c in delete_null_predict.columns]).show()

+---+---------+---------+-----------+---------------+----+--------+-----------+-----+---------+---------+----------+--------------+----------+-------------------+-------+----------+----+-------+-----+--------+----------+---+-----------+----------------+--------------+----------------------+----------------------+--------+-------+---------+----------+---------+-------------------+---------+-------------+-----------+---------------+---------------+-------------+------------------+----------+-----------------------+------------+---------------+------------+------------------+--------------------------+---------------------------+-----------+----------------+----------------+---------+-------------------------+--------------+------------+-----------+-----------+----------+-----------+-----------+------------------+------------+-------+-----------+-----------+----------+--------+----------------+------+-----------+------------+-----------+
| id|member_id|loan_amnt|funded_amnt|funded_amnt_in

In [ ]:
delete_null_predict.show()

+-------+---------+---------+-----------+---------------+----------+--------+-----------+-----+---------+--------------------+----------+--------------+----------+-------------------+----------+----------+--------------------+------------------+--------------------+--------+----------+-----+-----------+----------------+--------------+----------------------+----------------------+--------+-------+---------+----------+---------+-------------------+---------+-------------+-----------+---------------+---------------+-------------+------------------+----------+-----------------------+------------+---------------+------------+------------------+--------------------------+---------------------------+-----------+----------------+----------------+---------+-------------------------+--------------+------------+-----------+-----------+----------+-----------+-----------+------------------+------------+-------+-----------+-----------+----------+--------+----------------+------+-----------+---------

# Dealing with n/a in int column

In [ ]:
filter_mths_since_last_delinq = delete_null_predict.filter(delete_null_predict.mths_since_last_delinq != "n/a")

In [ ]:
from pyspark.sql.types import IntegerType
#convert it first to int
filter_mths_since_last_delinq = filter_mths_since_last_delinq.withColumn("mths_since_last_delinq", filter_mths_since_last_delinq["mths_since_last_delinq"].cast(IntegerType()))

In [ ]:
from pyspark.sql.functions import mean as _mean

#find the mean
df_stats = filter_mths_since_last_delinq.select(
    _mean(col('mths_since_last_delinq')).alias('mean')
).collect()

mean = df_stats[0]['mean']
round(mean)


34

In [ ]:
#when the value is "n/a" then replace it with the mean
#just testing, will not use this column bcs the null percentage is higher than 50%
targetDf = delete_null_predict.withColumn("mths_since_last_delinq", when(delete_null_predict["mths_since_last_delinq"] =="n/a", round(mean)).otherwise(delete_null_predict["mths_since_last_delinq"]))

In [ ]:
targetDf.show()

+-------+---------+---------+-----------+---------------+----------+--------+-----------+-----+---------+--------------------+----------+--------------+----------+-------------------+----------+----------+--------------------+------------------+--------------------+--------+----------+-----+-----------+----------------+--------------+----------------------+----------------------+--------+-------+---------+----------+---------+-------------------+---------+-------------+-----------+---------------+---------------+-------------+------------------+----------+-----------------------+------------+---------------+------------+------------------+--------------------------+---------------------------+-----------+----------------+----------------+---------+-------------------------+--------------+------------+-----------+-----------+----------+-----------+-----------+------------------+------------+-------+-----------+-----------+----------+--------+----------------+------+-----------+---------

In [ ]:
data_df = targetDf.withColumn("mths_since_last_delinq", targetDf["mths_since_last_delinq"].cast(IntegerType()))

In [ ]:
data_df.dtypes

[('id', 'int'),
 ('member_id', 'int'),
 ('loan_amnt', 'int'),
 ('funded_amnt', 'int'),
 ('funded_amnt_inv', 'double'),
 ('term', 'string'),
 ('int_rate', 'double'),
 ('installment', 'double'),
 ('grade', 'string'),
 ('sub_grade', 'string'),
 ('emp_title', 'string'),
 ('emp_length', 'string'),
 ('home_ownership', 'string'),
 ('annual_inc', 'string'),
 ('verification_status', 'string'),
 ('issue_d', 'string'),
 ('pymnt_plan', 'string'),
 ('desc', 'string'),
 ('purpose', 'string'),
 ('title', 'string'),
 ('zip_code', 'string'),
 ('addr_state', 'string'),
 ('dti', 'string'),
 ('delinq_2yrs', 'string'),
 ('earliest_cr_line', 'string'),
 ('inq_last_6mths', 'string'),
 ('mths_since_last_delinq', 'int'),
 ('mths_since_last_record', 'string'),
 ('open_acc', 'string'),
 ('pub_rec', 'string'),
 ('revol_bal', 'string'),
 ('revol_util', 'string'),
 ('total_acc', 'string'),
 ('initial_list_status', 'string'),
 ('out_prncp', 'string'),
 ('out_prncp_inv', 'string'),
 ('total_pymnt', 'string'),
 ('tota

In [ ]:
selectedColumns = data.select("id","member_id","loan_amnt","term","int_rate","installment","grade","emp_title","emp_length","home_ownership","annual_inc","verification_status","purpose")

In [ ]:
selectedColumns.show()

+-------+---------+---------+----------+--------+-----------+-----+--------------------+----------+--------------+----------+-------------------+------------------+
|     id|member_id|loan_amnt|      term|int_rate|installment|grade|           emp_title|emp_length|home_ownership|annual_inc|verification_status|           purpose|
+-------+---------+---------+----------+--------+-----------+-----+--------------------+----------+--------------+----------+-------------------+------------------+
|1077501|  1296599|     5000| 36 months|   10.65|     162.87|    B|                 n/a| 10+ years|          RENT|     24000|           Verified|       credit_card|
|1077430|  1314167|     2500| 60 months|   15.27|      59.83|    C|               Ryder|  < 1 year|          RENT|     30000|    Source Verified|               car|
|1077175|  1313524|     2400| 36 months|   15.96|      84.33|    C|                 n/a| 10+ years|          RENT|     12252|       Not Verified|    small_business|
|1076863| 

In [ ]:
selectedColumns.dtypes

[('id', 'int'),
 ('member_id', 'int'),
 ('loan_amnt', 'int'),
 ('term', 'string'),
 ('int_rate', 'double'),
 ('installment', 'double'),
 ('grade', 'string'),
 ('emp_title', 'string'),
 ('emp_length', 'string'),
 ('home_ownership', 'string'),
 ('annual_inc', 'string'),
 ('verification_status', 'string'),
 ('purpose', 'string')]

In [ ]:
amount_missing_df = selectedColumns.select([((count(when(col(c) == "n/a", c))/selectedColumns.count())*100).alias(c) for c in selectedColumns.columns])
amount_missing_df.show()

+---+---------+---------+----+--------+-----------+-----+-----------------+-----------------+--------------+----------+-------------------+--------------------+
| id|member_id|loan_amnt|term|int_rate|installment|grade|        emp_title|       emp_length|home_ownership|annual_inc|verification_status|             purpose|
+---+---------+---------+----+--------+-----------+-----+-----------------+-----------------+--------------+----------+-------------------+--------------------+
|0.0|      0.0|      0.0| 0.0|     0.0|        0.0|  0.0|5.776260588876466|5.030672839787422|           0.0|       0.0|                0.0|1.168266607786029...|
+---+---------+---------+----+--------+-----------+-----+-----------------+-----------------+--------------+----------+-------------------+--------------------+



In [ ]:
fixEmpTitle = selectedColumns.withColumn("emp_title", when((selectedColumns["emp_title"] == "n/a") & (selectedColumns["emp_length"] == "n/a"), lit("unemployed")).otherwise(selectedColumns["emp_title"]))

In [ ]:
fixEmpTitle.show()

+-------+---------+---------+----------+--------+-----------+-----+--------------------+----------+--------------+----------+-------------------+------------------+
|     id|member_id|loan_amnt|      term|int_rate|installment|grade|           emp_title|emp_length|home_ownership|annual_inc|verification_status|           purpose|
+-------+---------+---------+----------+--------+-----------+-----+--------------------+----------+--------------+----------+-------------------+------------------+
|1077501|  1296599|     5000| 36 months|   10.65|     162.87|    B|                 n/a| 10+ years|          RENT|     24000|           Verified|       credit_card|
|1077430|  1314167|     2500| 60 months|   15.27|      59.83|    C|               Ryder|  < 1 year|          RENT|     30000|    Source Verified|               car|
|1077175|  1313524|     2400| 36 months|   15.96|      84.33|    C|                 n/a| 10+ years|          RENT|     12252|       Not Verified|    small_business|
|1076863| 